In [1]:
import numpy as np
from glob import glob
from tqdm import tqdm
import h5py
import nrrd
import os
import pandas as pd
from dataset_split import remove_files

output_size =[128, 128, 64]

# 数组替换元素
def array_replace(array,olds,news):
    #olds:list of old value
    #news:list of new value
    olds = np.array(olds)
    news = np.array(news)
    offset = olds.max()*10
    tmps = olds+offset
    for old,tmp in zip(olds,tmps):
        array[array==old] = tmp
    for tmp,new in zip(tmps,news):
        array[array==tmp] = new
    return array

def covert_h5(glob_str, old_replaced, new_replaced):
    """
    备注：不要骨头，骨头合并到背景类别中
    """
    listt = glob(glob_str)
    #listt = glob('../../data/2018LA_Seg_Training Set/*/CTM.nrrd')
    #listt = glob('../../data/2018LA_Seg_Training Set/*/lgemri.nrrd')
    error_samples = []
    for item in tqdm(listt):
        sample_name = item.split('/')[-2]
        print(sample_name,':')#win系统改为'\\'
        
        image, img_header = nrrd.read(item)
        label, label_header = nrrd.read(item.replace(old_replaced, 'Segmentation-label.nrrd'))
        seg, seg_header = nrrd.read(item.replace(old_replaced, 'Segmentation.seg.nrrd'))
        
        # 灰度标准化
        image = (image-np.mean(image)) / np.std(image)
        image = image.astype(np.float32)
        
        # offset
        offset=[]
        for k in seg_header['Segmentation_ReferenceImageExtentOffset'].split():
            offset += [int(k)]
        sizes = seg_header['sizes'][1::]
        
        image = image[offset[0]:offset[0]+sizes[0],
                      offset[1]:offset[1]+sizes[1],
                      offset[2]:offset[2]+sizes[2]]
        label = label[offset[0]:offset[0]+sizes[0],
                      offset[1]:offset[1]+sizes[1],
                      offset[2]:offset[2]+sizes[2]].astype(np.uint8)
        
        # 错误病例：标记的尺寸和image尺寸不同，缺少其中一个类别或者多个类别的标记
        if not np.unique(label).tolist()==[0,1,2,3]:
            error_samples.append(sample_name)
            print("error sample(no dura/SC):",sample_name)
            continue
        
        if not image.shape==seg.shape[1::]:
            error_samples.append(sample_name)
            print("error sample(shape mismatch):",sample_name)
            continue
            
        # 类别名称和顺序
        target_name = ['dura','bone','SC']#目标类别顺序
        label_name = [
            seg_header['Segment0_Name'],
            seg_header['Segment1_Name'],
            seg_header['Segment2_Name'] 
            ]#人工标注的类别顺序
        ## 调整顺序，注意：seg是onehot编码
        idx = [label_name.index(name) for name in target_name]
        idx = np.array(idx)+1
        idx = [0]+idx.tolist()
        label = array_replace(label,olds=[0,1,2,3],news=idx)
        
        # 合并骨头到背景中
        label[label==2] = 0
        label[label==3] = 2
        
        print("image.shape:",image.shape)
        print("label.shape:",label.shape)
        assert np.unique(label).tolist()==[0,1,2], print('pixel classes are not [0,1,2]') 

        tempL = np.nonzero(label)
        minx, maxx = np.min(tempL[0]), np.max(tempL[0])
        miny, maxy = np.min(tempL[1]), np.max(tempL[1])
        minz, maxz = np.min(tempL[2]), np.max(tempL[2])
        w, h, d = label.shape
        px = max(output_size[0] - (maxx - minx), 0) // 2
        py = max(output_size[1] - (maxy - miny), 0) // 2
        minx = max(minx - np.random.randint(10, 20) - px, 0)
        maxx = min(maxx + np.random.randint(10, 20) + px, w)
        miny = max(miny - np.random.randint(10, 20) - py, 0)
        maxy = min(maxy + np.random.randint(10, 20) + py, h)
        
        image = image[minx:maxx, miny:maxy]
        label = label[minx:maxx, miny:maxy]
        print(label.shape)
        
        f = h5py.File(item.replace(old_replaced, new_replaced), 'w')
        f.create_dataset('image', data=image, compression="gzip")
        f.create_dataset('label', data=label, compression="gzip")
        f.close()
    print("total number of seg-samples:", len(listt))
    return error_samples

def covert_h5_unseg(glob_str, old_replaced, new_replaced):
    """
    备注：无标注数据的格式转换
    """
    listt = glob(glob_str)
    for item in tqdm(listt):
        sample_name = item.split('/')[-2]
        print(sample_name,':')#win系统改为'\\'
        # 读取原始图
        image, img_header = nrrd.read(item)     
#         # 缩小图像
#         image = image[0:-1:2,0:-1:2,:]
        # 灰度标准化
        image = (image - np.mean(image)) / np.std(image)
        image = image.astype(np.float32)
        print(image.shape)
        
        f = h5py.File(item.replace(old_replaced, new_replaced), 'w')
        f.create_dataset('image', data=image, compression="gzip")
        f.close()  
    print("total number of unseg-samples:", len(listt))

    
if __name__ == '__main__':
    # 有标签数据
    print('seg dataset:')
    ## 先删除旧文件
    dataset_dir = '../../data/CTM_dataset/Segmented'
    re = os.path.join(dataset_dir,'*/mri_norm2.h5')
    remove_files(re=re)
    ## 再生成新文件
    glob_str = '../../data/CTM_dataset/Segmented/*/CTM.nrrd'
    error_samples=covert_h5(glob_str,'CTM.nrrd','mri_norm2.h5')
    
    # 无标签数据
    print('unseg dataset:')
    ## 先删除旧文件
    dataset_dir = '../../data/CTM_dataset/unSegmented'
    re = os.path.join(dataset_dir,'*/mri_norm2.h5')
    remove_files(re=re)
    ## 再生成新文件
    glob_str = '../../data/CTM_dataset/unSegmented/*/CTM.nrrd'
    covert_h5_unseg(glob_str,'CTM.nrrd','mri_norm2.h5')
    glob_str = '../../data/CTM_dataset/unSegmented/*/CT-vol.nrrd'
    covert_h5_unseg(glob_str,'CT-vol.nrrd','mri_norm2.h5')      


  0%|          | 0/181 [00:00<?, ?it/s]

seg dataset:
845929 :
image.shape: (512, 381, 92)
label.shape: (512, 381, 92)
(156, 158, 92)


  1%|          | 1/181 [00:02<07:16,  2.42s/it]

1559027 :
image.shape: (512, 480, 73)
label.shape: (512, 480, 73)
(151, 200, 73)


  1%|          | 2/181 [00:04<07:08,  2.39s/it]

...

total number of unseg-samples: 212
1279071 :
(512, 512, 72)


 50%|█████     | 1/2 [00:01<00:01,  1.96s/it]

1755610 :
(512, 512, 85)


100%|██████████| 2/2 [00:04<00:00,  2.19s/it]

total number of unseg-samples: 2


# 数据集划分&生成*.list文件

In [2]:
from dataset_split import dataset_split, make_dataset_list
# 有标签数据
save_dir = '../../data/CTM_dataset'
dataset_dir = '../../data/CTM_dataset/Segmented'
list_train,list_test = dataset_split(path=dataset_dir,save_dir=save_dir)
# 无标签数据
dataset_dir = '../../data/CTM_dataset/unSegmented'
make_dataset_list(path=dataset_dir,save_dir=save_dir)

In [ ]:
set(list_train).interaction(set(error_samples))